# Map

![domain decomposition](https://computing.llnl.gov/tutorials/parallel_comp/images/domain_decomp.gif)


## `map` function example

The `map(func, seq)` Python function applies the function func to all the elements of the sequence seq. It returns a new list with the elements changed by func

In [ ]:
def f(x):
    return x * x

rdd = [2, 6, -3, 7]
res = map(f, rdd )
res  # Res is an iterator

In [ ]:
print(*res)

In [ ]:
from operator import mul
rdd1, rdd2 = [2, 6, -3, 7], [1, -4, 5, 3]
res = map(mul, rdd1, rdd2 ) # element wise sum of rdd1 and rdd2 

In [ ]:
print(*res)

# Reduce

![MapReduce](http://mm-tom.s3.amazonaws.com/blog/MapReduce.png)

## `functools.reduce` example

The function `reduce(func, seq)` continually applies the function func() to the sequence seq and return a single value. For example, reduce(f, [1, 2, 3, 4, 5]) calculates f(f(f(f(1,2),3),4),5).

In [ ]:
from functools import reduce
from operator import add

reduce(mul, rdd) # computes ((((1+2)+3)+4)+5)

In [ ]:
p = 1
for x in rdd:
    p *= x
p

In [ ]:
def g(x,y):
    return x * y

reduce(g, rdd) 

In [ ]:
p = 1
for x in rdd:
    p *= x
p

## Weighted mean and Variance

If the generator of random variable $X$ is discrete with probability mass function $x_1 \mapsto p_1, x_2 \mapsto p_2, \ldots, x_n \mapsto p_n$ then

$$\operatorname{Var}(X) = \left(\sum_{i=1}^n p_i x_i ^2\right) - \mu^2,$$

where $\mu$ is the average value, i.e.

$$\mu = \sum_{i=1}^n p_i x_i. $$

In [ ]:
X = [5, 1, 2, 3, 1, 2, 5, 4]
P = [0.05, 0.05, 0.15, 0.05, 0.15, 0.2, 0.1, 0.25]

In [ ]:
for x, p in zip(X, P):
    print(f" x = {x} ..... p = {p}")

In [ ]:
from itertools import zip_longest, cycle

for x, p in zip_longest(X, [0.1], fillvalue=0.0):
    print(f" x = {x} ..... p = {p}")
    
    

In [ ]:
sum(P)

### Exercise 5.1

- Write functions to compute the average value and variance using for loops

In [ ]:
def weighted_mean( X, P):
    return sum([x*p for x, p in zip(X,P)])

weighted_mean(X,P)

In [ ]:
def variance(X, P):
    mu = weighted_mean(X,P)
    return sum([p*x*x for x,p in zip(X,P)]) - mu**2

variance(X, P)

### Exercise 5.2

- Write functions to compute the average value and variance using `map` and `reduce`

In [ ]:
from operator import add, mul
from functools import reduce

def weighted_mean( X, P):
    return reduce(add,map(mul, X, P))

weighted_mean(X,P)

In [ ]:
def variance(X, P):
    mu = weighted_mean(X,P)
    return reduce(add,map(lambda x,p:p*x*x, X, P)) - mu**2

variance(X, P)

In [ ]:

P

In [ ]:
res = filter( lambda p: p > 0.1, P)
print(*res)

In [ ]:
res = filter( lambda x: x % 3 == 0, range(15))
print(*res)

*NB: Exercises above are just made to help to understand map-reduce process.
This is a bad way to code a variance in Python. You should use [Numpy](http://www.numpy.org) instead.*

## Wordcount 

We will modify the `wordcount` application into a map-reduce process.

The `map` process takes text files as input and breaks it into words. The `reduce`  process sums the counts for each word and emits a single key/value with the word and sum.

We need to split the wordcount function we wrote in notebook 04 in order to use map and reduce. 

In the following exercices we will implement in Python the Java example described in [Hadoop documentation](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html#Example:_WordCount_v1.0).

# Map 

## Read file and return a key/value pairs

### Exercise 5.3

Write a function `mapper` with a single file name as input that returns a sorted sequence of tuples (word, 1) values.

```pybt
mapper('sample.txt')
[('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('adipisci', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('aliquam', 1), ('amet', 1), ('amet', 1), ('amet', 1)...
```

In [ ]:
import lorem
with open('sample.txt','w') as f:
    f.write(lorem.text())

In [ ]:
def mapper(filename):
    with open(filename) as f:
        data = f.read()
        
    data = data.strip().replace(".","").lower().split()
        
    return sorted([(w,1) for w in data])

mapper("sample.txt")

# Partition

### Exercise 5.4

Create a function named `partitioner` that stores the key/value pairs from `mapper`  that group (word, 1) pairs into a list as:
```python
partitioner(mapper('sample.txt'))
[('adipisci', [1, 1, 1, 1, 1, 1, 1]), ('aliquam', [1, 1, 1, 1, 1, 1, 1]), ('amet', [1, 1, 1, 1],...]
```

In [ ]:
from collections import defaultdict


def partitioner(mapped_values):
    
    res = defaultdict(list)
    for w, c in mapped_values:
        res[w].append(c)
        
    return res.items()


partitioner(mapper('sample.txt'))

# Reduce 

## Sums the counts and returns a single key/value (word, sum).

### Exercice 5.5

Write the function `reducer` that read a tuple (word,[1,1,1,..,1]) and sum the occurrences of word to a final count, and then output the tuple (word,occurences).

```python
reducer(('hello',[1,1,1,1,1])
('hello',5)
```

In [91]:
from operator import itemgetter


def reducer( item ):
    w, v = item
    return (w,len(v))

reducer(('hello',[1,1,1,1,1]))

('hello', 5)

# Process several files

Let's create 8 files `sample[0-7].txt`. Set most common words at the top of the output list.

In [92]:
from lorem import text
for i in range(8):
    with open("sample{0:02d}.txt".format(i), "w") as f:
        f.write(text())

In [93]:
import glob
files = sorted(glob.glob('sample0*.txt'))
files

['sample00.txt',
 'sample01.txt',
 'sample02.txt',
 'sample03.txt',
 'sample04.txt',
 'sample05.txt',
 'sample06.txt',
 'sample07.txt']

### Exercise 5.6
- Use functions implemented above to count (word, occurences) by using a for loops over files and partitioned data.

In [ ]:
from itertools import chain

def wordcount(files):
    
    mapped_values = [mapper(file) for file in files]
    
    partioned_values = partitioner(chain(*mapped_values))
            
    return sorted([reducer(val) for val in partioned_values],
                  key = itemgetter(1),
                  reverse = True)

wordcount(files)

### Exercise 5.7
- This time use `map` function to apply mapper and reducer.


In [100]:
def wordcount(files):
    
    mapped_values = map(mapper, files)
    partioned_values = partitioner(chain(*mapped_values))
    
    return sorted( map(reducer, partioned_values),
                 key=itemgetter(1),
                 reverse=True)
    
    
wordcount(files)    

[('voluptatem', 69),
 ('amet', 67),
 ('sit', 66),
 ('dolorem', 65),
 ('non', 61),
 ('quisquam', 60),
 ('sed', 59),
 ('adipisci', 55),
 ('consectetur', 55),
 ('magnam', 54),
 ('velit', 54),
 ('est', 53),
 ('ipsum', 53),
 ('neque', 53),
 ('numquam', 53),
 ('eius', 52),
 ('quaerat', 52),
 ('tempora', 50),
 ('dolor', 49),
 ('dolore', 48),
 ('etincidunt', 48),
 ('modi', 47),
 ('aliquam', 45),
 ('porro', 45),
 ('quiquia', 45),
 ('labore', 42),
 ('ut', 42)]